# 8-K form #
  
The notebook has three functions to parse 8-K forms. The first, *get_text()*, gets all the text from the form and returns a string. It takes the link to the form as an argument. Second, *get_items()* returns a list containing all items mentioned in the form. It takes a string as input. Finally, *get_text_item()* returns a data frame with Company name, CIK, Item, and text.  It takes a link and a string (e.g. "Item 6") as an argument.

**Caveats:** 
Not all forms have a clear end to the text item. The script, then, takes everything until signature. Expect to do some cleaning after retrieving the content.  
The functions below are parsers. You need to feed a SEC link into it. There are many python and r packages to get a direct link to the fillings.

In [ ]:
# required
import re
import requests
from bs4 import BeautifulSoup as bs

In [ ]:
def get_text(link):
    page = requests.get(link)
    html = bs(page.content, "lxml")
    text = html.get_text().replace(u'\xa0', ' ').split("\n")
    return(text)

def get_items(text):
    item_pattern = re.compile("(Item\s\d[\.\d]*)")
    value = list()
    text = str(text)
    for i in item_pattern.findall(text):
        i = re.sub("\.$", "", i)
        i = i.strip()
        if i not in value:
            value.append(i)
    return(value)

def get_text_item(link, item):
    page = requests.get(link)
    html = bs(page.content, "lxml")
    text = html.get_text().replace(u'\xa0', ' ').replace("\t", " ").replace("\x92", "'").split("\n")
    company_cik = re.compile("(CENTRAL INDEX KEY:)([\s\d]+)")
    company_name = re.compile("(COMPANY CONFORMED NAME:)(.+)")
    item_pattern = re.compile(item, re.IGNORECASE)
    stop = re.compile("SIGNATURE", re.IGNORECASE)
    k8_text = ""
    data_dict = dict()
    control = 0
    for line in text:
        if company_cik.search(line) is not None:
            data_dict["cik"] = company_cik.search(line, re.IGNORECASE).group(2).strip()
        elif company_name.search(line) is not None:
            data_dict["conm"] = company_name.search(line).group(2).strip()
        elif item_pattern.search(line) is not None:
            k8_text = k8_text + " " + line
            control = 1
        elif control == 1:
            if stop.search(line) is not None:
                data_dict["text"] = k8_text
                control = 0
            else:
                k8_text = k8_text + " " + line
        else:
            pass
    data = pd.DataFrame.from_dict([data_dict])
    return(data)  